In [2]:
import numpy as np
import urllib
import os

# Download the dataset

In [3]:
# Get the webpage
url = 'https://landsat.usgs.gov/landsat-8-cloud-cover-assessment-validation-data'
response = urllib.request.urlopen(url)

# Find the data urls in the source
data_urls = []
for line in response:
    if 'https://landsat.usgs.gov/cloud-validation/cca' in str(line) and 'tar' in str(line):        
        data_urls.append(str(line)[47:124])

print("Products found:", np.size(data_urls))
print("Example url:   ", data_urls[0])        

Products found: 96
Example url:    https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz


In [4]:
# define backgrounds
backgrounds = ['Barren',
               'Forest',
               'GrassCrops',
               'Shrubland',
               'SnowIce',
               'Urban',
               'Water',
               'Wetlands']

In [ ]:
# get test dataset
index = 0
for i in range(1):
    !wget {data_urls[i]} -P {'./tar/' + backgrounds[index] + '/'}

[0] Downloading 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz' ...
Saving './tar/Barren//LC80420082013220LGN00.tar.gz'
^C
Just got 177483815 of 914473396 bytes (https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz)
HTTP response 200 OK [https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz]
Unexpected body length 177483815.


In [36]:
from IPython.display import clear_output

last_percent_decimal_step = 0.0

def rep_hoo(current_block_count, block_size, total_size):
    percentage = current_block_count*block_size/total_size*100
    
    global last_percent_decimal_step 
    if abs(percentage - last_percent_decimal_step) >= .1:
        last_percent_decimal_step = percentage
        print(f"Download at {percentage:.1f}% - {current_block_count} blocks of {total_size//block_size} total blocks")
        clear_output(wait=True)
    

In [37]:
import urllib.request

index = 0 # index -1
for i in range(1, np.size(data_urls)): # range(0, np.size(data_urls))
    if i%12 == 0:  # There are 12 scenes in every background category       
        index += 1     
    (txt, httpmsg) = urllib.request.urlretrieve(data_urls[i], "./tar/"+backgrounds[index]+"/"+data_urls[i].split("/")[-1], reporthook=rep_hoo)

KeyboardInterrupt: 

In [ ]:
import asyncio
import subprocess


index = -1
bulk_size = 12
remaining_dls = 12 # np.size(data_urls)



loop = asyncio.get_event_loop()


async def download_tar_url(url, background):
    return await subprocess.Popen(f"wget {url} -P {'./tar/' + background + '/'}")
    #return await os.system(f"wget {url} -P {'./tar/' + background + '/'}")

async def download_bulk(bulk_size):
    for i in range(bulk_size):
        index+=1
        download_tar_url(data_urls[index+i], background=backgrounds[index%12])
    
async def download_all():
    while remaining_dls > 0:
        await download_bulk(bulk_size=bulk_size)
        remaining_dls -= bulk_size

asyncio.ensure_future(download_all())
loop.run_forever()

In [ ]:
# Download all the products
index = -1
for i in range(0, np.size(data_urls)):
    if i%12 == 0:  # There are 12 scenes in every background category       
        index += 1     
    !wget {data_urls[i]+'.tar.gz'} -P {'./tar/' + backgrounds[index] + '/'}

http://www.gnu.org/software/parallel/man.html#example__download_10_images_for_each_of_the_past_30_days

Running 10000 jobs with 100 in parallel:
> seq 10000 | parallel -j100 wget https://www.example.com/page{}.html

In [ ]:
# Download all the products in background
index = -1
for i in range(0, np.size(data_urls)):
    if i%12 == 0:  # There are 12 scenes in every background category       
        index += 1
    !wget -b {data_urls[i]+'.tar.gz'} -P {'./tar/' + backgrounds[index] + '/'}

In [ ]:
folders = [i for i in os.listdir("./tar/") if i not in [".gitignore", ".gitkeep"]]

!mkdir {"unzipped"}
for folder in folders:
    !mkdir {"./unzipped/" + folder}
    files = sorted(os.listdir("tar/" + folder))
    files = [f for f in files if '.tar.gz' in f] 
    for file in files:
        try: 
            print("----")
            print("Unzipping product:", file)
            !tar -xvzf {"./tar/" + folder + '/' + file} -C {'./unzipped/' + folder + '/'}
        except:
            print("----------------------------------------------------")
            print("UNZIP ERROR WITH PRODUCT:", file)
            pass

----
Unzipping product: LC80420082013220LGN00.tar.gz
BC/LC80420082013220LGN00/LC80420082013220LGN00_B10.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B11.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B1.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B2.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B3.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B4.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B5.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B6.TIF

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
